### Loading data from several spreadsheets, note many cell outputs in this notebook are redacted to avoid displaying personalized info

In [70]:
import pandas as pd
import spacy
from collections import defaultdict
import re


# Load SpaCy's English NLP model
nlp = spacy.load('en_core_web_sm')

import pandas as pd

# Paths to the Excel files
file_paths = ['/Users/adamhunter/Documents/renewablydocs/um_ghg_data/21-23 inventory reference/air and bus travel/business travel/FY21 Airline Spend.xlsx',
              '/Users/adamhunter/Documents/renewablydocs/um_ghg_data/21-23 inventory reference/air and bus travel/business travel/FY22 Airline Spend.xlsx',
              '/Users/adamhunter/Documents/renewablydocs/um_ghg_data/21-23 inventory reference/air and bus travel/business travel/FY23 Airline Spend.xlsx',
              '/Users/adamhunter/Downloads/Book1.xlsx']

# # Initialize a list to store the dataframes
dfs = []

df = pd.read_excel(file_paths[3])
# df = df[(df['Campus'] == 3) | (df['Campus'] == 1)]
dfs.append(df)


In [71]:
df.columns
df['INV DATE'] = pd.to_datetime(df['INV DATE'])
# df = df[df['INV DATE'] <= pd.Timestamp('2023-08-23')]


### Checking which campus accounts are contributing the most

In [73]:
import plotly.express as px

# Combine the amounts for campus numbers 5 and 6
df['Campus'] = df['Campus'].replace({6: 5})

# Now group by 'Campus' and sum the 'AMOUNT'
campus_amount_sum = df.groupby('Campus')['AMOUNT'].sum().reset_index()

# Update the campus names dictionary
campus_names = {1: 'Forestry Conservation', 3: 'Missoula', 4: 'Helena', 5: 'Butte', 8: 'Dillon'}

campus_amount_sum = campus_amount_sum.sort_values(by='AMOUNT', ascending=False)

# Map the campus numbers to names for a more informative chart
campus_amount_sum['Campus'] = campus_amount_sum['Campus'].map(campus_names)

# Create a histogram using Plotly
total_amount = campus_amount_sum['AMOUNT'].sum()
campus_amount_sum['Percentage'] = (campus_amount_sum['AMOUNT'] / total_amount) * 100
fig = px.bar(campus_amount_sum, x='Campus', y='AMOUNT', title='Sum of Air Travel Spend by Campus',
             text=campus_amount_sum['Percentage'].apply(lambda x: '{:.2f}%'.format(x)))

# Where `threshold` is a value you determine is too small to fit the text inside the bars

# Show the plot
fig.show()

import plotly.io as pio

# Save the figure to the Downloads folder with a high-quality rendering
fig_path = '/Users/adamhunter/Downloads/air_travel_spend_by_campus.png'
fig.update_layout(width=1200, height=675)  # Adjusted for a wider aspect ratio at half the overall size
pio.write_image(fig, fig_path)
print(f"Chart saved to {fig_path}")
print(total_amount)

Chart saved to /Users/adamhunter/Downloads/air_travel_spend_by_campus.png
4897794.89


In [75]:
# Save the figure to the Downloads folder with a high-quality rendering
fig_path = '/Users/adamhunter/Downloads/air travel spend by campus.png'
fig.update_layout(width=1200, height=675)  # Adjusted for a wider aspect ratio at half the overall size
pio.write_image(fig, fig_path)
print(f"Chart saved to {fig_path}")
print(total_amount)

Chart saved to /Users/adamhunter/Downloads/air travel spend by campus.png
4897794.89


### Functions to extract faculty/staff names from air travel descriptions, a necessary first step in analyzing which departments are doing the most air travel. Lots of experimentation here to find a set of rules that would work with the varied hand-filled description styles.

In [ ]:

# Function to clean and split description
def clean_and_split_description(description, remove_words):
    # Convert description to string and replace non-alphanumeric characters with spaces
    description = re.sub(r'\W+', ' ', str(description))
        
    # Remove specific words, case insensitive
    for word in remove_words:
        description = re.sub(r'\b'+word+r'\b', '', description, flags=re.IGNORECASE)
        
    # Split the description into words
    words = description.split()
    
    return words

# Function to extract last name from words
def extract_last_name_from_words(words, last_names):
    # Initialize the last name for this description as None
    last_name = None
        
    # Go through all words
    for word in words:
        # Check if the word starts with a capital letter or is all caps
        if word[0].isupper():
            # Check if only the first and last letters are capitalized
            if len(word) > 1 and word[1:-1].islower() and word[-1].isupper():
                # Remove the last character
                word = word[:-1]
                
            # Convert the word to lower case
            word = word.lower()
                
            # Update the dictionary
            last_names[word] += 1

            last_name = word
                
            # Break the loop
            break
    
    return last_name, last_names

# Function to extract last names from descriptions
def extract_last_names(df):
    # Extract the descriptions
    descriptions = df['COMM DESC']  # Extracting the 'COMM DESC' column as descriptions

    # Initialize a dictionary to store last names and their frequencies
    last_names = defaultdict(int)

    # List of words to remove
    remove_words = ['purchase', 'airfare', 'card', 'transaction', 'flight']

    # List to store the extracted last names
    extracted_last_names = []

    # Go through the descriptions
    for description in descriptions:
        words = clean_and_split_description(description, remove_words)
        last_name, last_names = extract_last_name_from_words(words, last_names)
        
        # Add the last name to the list
        extracted_last_names.append(last_name)

    # Add the extracted last names as a new column in the DataFrame
    df['extracted_last_name'] = extracted_last_names

    # Now, last_names is a dictionary where the keys are last names and the values are their frequencies
    return df, last_names

# Apply the function to all dataframes in the list
for i in range(len(dfs)):
    dfs[i], last_names = extract_last_names(dfs[i])

dfs[0]

In [24]:
df['AMOUNT'].sum()

4310969.59

### Calculating totals by extracted last name

In [25]:
# Initialize a list to store the grouped dataframes
grouped_dfs = []

# Iterate over all dataframes in 'dfs'
for df in dfs:
    # Group by 'last_name' and sum the 'AMOUNT' column
    grouped_df = df.groupby('extracted_last_name')['AMOUNT'].sum().reset_index()

    # Sort the DataFrame by 'AMOUNT' in descending order
    grouped_df = grouped_df.sort_values('AMOUNT', ascending=False)

    # Append the grouped dataframe to the list
    grouped_dfs.append(grouped_df)


### Redacted cells below displaying 'top spenders'

In [ ]:
# Import plotly.express for data visualization
import plotly.express as px

# Iterate over all dataframes in 'grouped_dfs'
for i, df in enumerate(grouped_dfs):
    # Create a horizontal bar chart using plotly for each dataframe
    fig = px.bar(df.head(10), 
                 x='AMOUNT', 
                 y='extracted_last_name', 
                 orientation='h', 
                 title=f'Top 10 Spenders in {2023+i}',
                 labels={'extracted_last_name':'Last Name', 'AMOUNT':'Total Spent'})
    # Display the plot
    fig.show()

In [ ]:
# Concatenate the dataframes in grouped_dfs into a single dataframe
concat_df = pd.concat(grouped_dfs)

# Group by 'last_name' and sum the 'AMOUNT' column
total_df = concat_df.groupby('extracted_last_name')['AMOUNT'].sum().reset_index()

# Sort the DataFrame by 'AMOUNT' in descending order
total_df = total_df.sort_values('AMOUNT', ascending=False)

# Create a horizontal bar chart using plotly for the top 10 last names
fig = px.bar(total_df.head(15), 
             x='AMOUNT', 
             y='extracted_last_name', 
             orientation='h', 
             title='Top 15 Spenders',
             labels={'extracted_last_name':'Last Name', 'AMOUNT':'Amount Spent'})

# Display the plot
fig.show()


In [28]:
print(len(total_df))

951


In [ ]:
# Save 'total_df' DataFrame to a CSV file
total_df.to_csv('total_df.csv', index=False)


### Beginning of webscraping section, the online UM directory was scraped to connect faculty/staff names with departments. In cases where last names are shared by members of multiple departments, all possible departments are returned for that name.

In [ ]:
from bs4 import BeautifulSoup
import requests
import time

# Function to scrape departments for a given name
def scrape_departments(name):
    # Print a message
    print(f'Scraping {name}')

    # URL of the online directory
    url = f"https://apps.umt.edu/directory/advanced?first_name=&last_name={name}&e=1#search_by_name"

    # Send a GET request to the server
    response = requests.get(url)

    # Parse the HTML content of the page with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Check if no results were found
    if "No results found for" in soup.text:
        departments = []
    else:
        # Find all departments in the page
        departments = [td.text.strip() for td in soup.find_all('td', attrs={'class': 'department'})]

    # Wait for 2 seconds
    time.sleep(2)

    return departments, 'yes'

# Add a 'scraped' column to the DataFrame if it doesn't exist
if 'scraped' not in total_df.columns:
    total_df['scraped'] = 'no'

# Add a 'departments' column to the DataFrame if it doesn't exist
if 'departments' not in total_df.columns:
    total_df['departments'] = None

# Sort the DataFrame by the 'scraped' column in-place
total_df.sort_values('scraped', inplace=True)

# Apply the function to the rows of the DataFrame that haven't been scraped yet
for i, row in total_df.iterrows():
    if row['scraped'] == 'no':
        total_df.at[i, 'departments'], total_df.at[i, 'scraped'] = scrape_departments(row['extracted_last_name'])

In [160]:
total_df_copy.to_csv('total_df_copy.csv', index=False)


In [ ]:
# Filter the DataFrame to only include rows where 'departments' is null and 'scraped' is 'yes'
# Create a copy of the DataFrame
total_df_copy = total_df.copy()

# Filter the DataFrame copy to only include rows where 'departments' is null and 'scraped' is 'yes'
total_df['departments'] = total_df['departments'].apply(lambda d: '' if not d else d)

# Remove all instances in total_df with 'departments' list longer than 2
total_df = total_df[total_df['departments'].apply(lambda x: len(x) <= 2)]
# Remove instances where 'departments' is an empty string
total_df = total_df[total_df['departments'] != '']

# Change the value of departments for extracted last name 'edwards' to 'Mansfield Center Administration'
total_df.loc[total_df['extracted_last_name'] == 'edwards', 'departments'] = 'Mansfield Center Administration'


highlighted_df = total_df[(total_df['departments'] == '') & (total_df['scraped'] == 'yes')]

# Print the number of such rows
highlighted_count = len(highlighted_df)
print(f"Number of rows where 'departments' is null and 'scraped' is 'yes': {highlighted_count}")

# Display the filtered DataFrame
highlighted_df


In [2]:
import pandas as pd

# Load total_df from the specified file
total_df = pd.read_csv('/Users/adamhunter/Documents/renewablydocs/um_ghg_data/Eva commuter:air travel followups/total_df_copy.csv')


In [ ]:
total_df

### One major spender was being miscategorized, there are probably other cases of this, but this was a particularly large one.

In [6]:
# Change the value of departments for extracted last name 'edwards' to 'Mansfield Center Administration'
total_df.loc[total_df['extracted_last_name'] == 'edwards', 'departments'] = 'Mansfield Center Administration'

In [7]:
# Custom function to convert string representation of lists into actual lists
def convert_to_list(val):
    if pd.notnull(val):
        # Remove brackets and extra quotes, then split by comma and strip whitespaces
        return val.replace('[','').replace(']','').replace('\'', '').split(', ')
    else:
        return []

# Apply the function to the 'departments' column
total_df['departments'] = total_df['departments'].apply(convert_to_list)

# Now you can use .explode
exploded_df = total_df.explode('departments')

In [8]:
# Remove instances where 'extracted_last_name' is less than 3 characters
exploded_df = exploded_df[exploded_df['extracted_last_name'].str.len() >= 3]


In [205]:

# Group by 'departments' and sum the 'AMOUNT' column
department_totals = exploded_df.groupby('departments')['AMOUNT'].sum().reset_index()


In [206]:
department_totals

,departments,AMOUNT
0,"""Natl Native Childrens Trauma Ctr""",13384.74
1,"""OConnor Ctr for the Rocky Mtn West""",116477.16
2,"""Presidents Office""",55952.52
3,ACF Child & Family Center,1557.21
4,ASUM Childcare,6672.97
...,...,...
134,VP Enrollment Management,980.65
135,VPAA Dept Operations,18470.64
136,Veterans Education/Transition Srvs,7476.66
137,Vice Provost Student Success Admn,9797.69


In [207]:
# Sort the DataFrame by 'AMOUNT' in descending order
department_totals = department_totals.sort_values('AMOUNT', ascending=False)

### The final result of the analysis is here, the total amounts are just estimates due to the issues with faculty names being associated with multiple departments in some cases. However the general thrust of the analysis, that the top travellers are from athletics and the mansfield library, remains valid.

In [208]:
import plotly.express as px

# Create a horizontal bar chart
fig = px.bar(department_totals.head(10), 
             x='AMOUNT', 
             y='departments', 
             orientation='h', 
             title='Top 10 Departments by Total Amount',
             labels={'departments':'Department', 'AMOUNT':'Total Amount'})

# Display the plot
fig.show()